# Create and Load DB Schema

In [5]:
import psycopg2
from sql_queries import create_table_queries, drop_table_queries

In [6]:
# create connection
conn = psycopg2.connect("host=127.0.0.1 dbname=postgres user=postgres password=postgres")
conn.set_session(autocommit=True)
cur = conn.cursor()

In [7]:
# create sparkify database with UTF8 encoding
cur.execute("DROP DATABASE IF EXISTS sparkifydb")
cur.execute("CREATE DATABASE sparkifydb WITH ENCODING 'utf8' TEMPLATE template0")

In [8]:
# close connection to default database
conn.close() 

In [9]:
# open connection to newly created db
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=postgres password=postgres")
cur = conn.cursor()

In [10]:
# run drop statements
for query in drop_table_queries:
    cur.execute(query)
    conn.commit()

In [11]:
# run create statements 
for query in create_table_queries:
    cur.execute(query)
    conn.commit()

In [225]:
conn.close()


# ETL Process
# Process song_data

In [12]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [355]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=postgres password=postgres")
cur = conn.cursor()

In [65]:
# function to get list of all song JSON files in filepath

def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

In [66]:
song_files = get_files('data/song_data')

In [415]:
song_files

['/Users/ggranda/Dropbox/data_eng/Postgres/udacity_Data_Modeling_with_Postgres/data/song_data/A/A/A/TRAAAEF128F4273421.json',
 '/Users/ggranda/Dropbox/data_eng/Postgres/udacity_Data_Modeling_with_Postgres/data/song_data/A/A/A/TRAAARJ128F9320760.json',
 '/Users/ggranda/Dropbox/data_eng/Postgres/udacity_Data_Modeling_with_Postgres/data/song_data/A/A/A/TRAAAFD128F92F423A.json',
 '/Users/ggranda/Dropbox/data_eng/Postgres/udacity_Data_Modeling_with_Postgres/data/song_data/A/A/A/TRAAAPK128E0786D96.json',
 '/Users/ggranda/Dropbox/data_eng/Postgres/udacity_Data_Modeling_with_Postgres/data/song_data/A/A/A/TRAAABD128F429CF47.json',
 '/Users/ggranda/Dropbox/data_eng/Postgres/udacity_Data_Modeling_with_Postgres/data/song_data/A/A/A/TRAAAAW128F429D538.json',
 '/Users/ggranda/Dropbox/data_eng/Postgres/udacity_Data_Modeling_with_Postgres/data/song_data/A/A/A/TRAAADZ128F9348C2E.json',
 '/Users/ggranda/Dropbox/data_eng/Postgres/udacity_Data_Modeling_with_Postgres/data/song_data/A/A/A/TRAAAVG12903CFA543

In [68]:
# select the first song from list
filepath = song_files[0]

In [113]:
# read data and load into data frame
df = pd.DataFrame([pd.read_json(filepath, typ='series')])

In [114]:
type(df)

pandas.core.frame.DataFrame

In [115]:
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR7G5I41187FB4CE6C,None,None,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982


In [117]:
df.values

array([[1, 'AR7G5I41187FB4CE6C', None, None, 'London, England',
        'Adam Ant', 'SONHOTT12A8C13493C', 'Something Girls', 233.40363,
        1982]], dtype=object)

In [78]:
df.dtypes

num_songs             int64
artist_id            object
artist_latitude      object
artist_longitude     object
artist_location      object
artist_name          object
song_id              object
title                object
duration            float64
year                  int64
dtype: object

In [79]:
# assing dataframe values to variables
num_songs, artist_id, artist_latitude, artist_longitude, artist_location, artist_name, song_id, title, duration, year = df.values[0]

In [80]:
artist_name

'Adam Ant'

In [81]:
# select the relevant values and add it to a list
song_data = [song_id, title, artist_id, year, duration]
song_data

['SONHOTT12A8C13493C',
 'Something Girls',
 'AR7G5I41187FB4CE6C',
 1982,
 233.40363]

In [82]:
type(song_data)

list

In [83]:
print(song_data)

['SONHOTT12A8C13493C', 'Something Girls', 'AR7G5I41187FB4CE6C', 1982, 233.40363]


In [84]:
# insert Record into Song Table
cur.execute(song_table_insert, song_data)
conn.commit()

### ^^^ Fails due to ForeignKeyViolation
# Process artist data

In [85]:
# Select columns for artist ID, name, location, latitude, and longitude
df

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR7G5I41187FB4CE6C,None,None,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982


In [86]:
artist_data = [artist_id, artist_name, artist_location, artist_latitude, artist_longitude]

In [87]:
artist_data

['AR7G5I41187FB4CE6C', 'Adam Ant', 'London, England', None, None]

In [88]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

In [89]:
# now we can load song data
cur.execute(song_table_insert, song_data)
conn.commit()

# Process log_data
In this part, you'll perform ETL on the second dataset, log_data, to create the time and users dimensional tables, as well as the songplays fact table.

Let's perform ETL on a single log file and load a single record into each table.

* Use the get_files function provided above to get a list of all log JSON files in data/log_data
* Select the first log file in this list
* Read the log file and view the data

In [175]:
log_files = get_files('data/log_data')

In [176]:
len(log_files)

30

In [177]:
filepath = log_files[0]

In [178]:
filepath

'/Users/ggranda/Dropbox/data_eng/Postgres/udacity_Data_Modeling_with_Postgres/data/log_data/2018/11/2018-11-11-events.json'

In [185]:
df2 = pd.read_json(filepath, lines=True)

In [180]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 18 columns):
artist           73 non-null object
auth             95 non-null object
firstName        92 non-null object
gender           92 non-null object
itemInSession    95 non-null int64
lastName         92 non-null object
length           73 non-null float64
level            95 non-null object
location         92 non-null object
method           95 non-null object
page             95 non-null object
registration     92 non-null float64
sessionId        95 non-null int64
song             73 non-null object
status           95 non-null int64
ts               95 non-null int64
userAgent        92 non-null object
userId           95 non-null object
dtypes: float64(2), int64(4), object(12)
memory usage: 13.5+ KB


In [126]:
df2.describe()

,itemInSession,length,registration,sessionId,status,ts
count,95.000000,73.000000,9.200000e+01,95.000000,95.000000,9.500000e+01
mean,4.010526,237.787905,1.540904e+12,423.357895,202.252632,1.541948e+12
std,4.006629,154.423459,1.371988e+08,63.833713,15.442390,1.645854e+07
min,0.000000,18.415870,1.540512e+12,120.000000,200.000000,1.541904e+12
25%,1.000000,189.622400,1.540794e+12,435.000000,200.000000,1.541939e+12
50%,3.000000,217.704040,1.540941e+12,437.000000,200.000000,1.541948e+12
75%,6.500000,259.839550,1.541020e+12,448.000000,200.000000,1.541962e+12
max,14.000000,1449.116280,1.541097e+12,480.000000,307.000000,1.541980e+12


In [295]:
df2.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,2018-11-11 02:33:56.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,2018-11-11 02:36:10.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,2018-11-11 02:40:34.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,2018-11-11 04:36:13.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
5,Matmos,Logged In,Joseph,M,0,Gutierrez,1449.11628,free,"Columbia, SC",PUT,NextSong,1.540809e+12,284,Supreme Balloon,200,2018-11-11 04:36:46.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3...",75


# 3: time Table
Extract Data for Time Table
* Filter records by NextSong action
* Convert the ts timestamp column to datetime
    * Hint: the current timestamp is in milliseconds
* Extract the timestamp, hour, day, week of year, month, year, and weekday from the ts column and set time_data to a list containing these values in order
    * Hint: use pandas' dt attribute to access easily datetimelike properties.
* Specify labels for these columns and set to column_labels
* Create a dataframe, time_df, containing the time data for this file by combining column_labels and time_data into a dictionary and converting this into a dataframe

In [190]:
# filtering by nextsong and converting ts column to date time
df2 = df2[df2['page'] == 'NextSong'].astype({'ts': 'datetime64[ms]'})

In [258]:
df2.dtypes

artist                   object
auth                     object
firstName                object
gender                   object
itemInSession             int64
lastName                 object
length                  float64
level                    object
location                 object
method                   object
page                     object
registration            float64
sessionId                 int64
song                     object
status                    int64
ts               datetime64[ns]
userAgent                object
userId                   object
dtype: object

In [192]:
df2.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,2018-11-11 02:33:56.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,2018-11-11 02:36:10.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,2018-11-11 02:40:34.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,2018-11-11 04:36:13.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
5,Matmos,Logged In,Joseph,M,0,Gutierrez,1449.11628,free,"Columbia, SC",PUT,NextSong,1.540809e+12,284,Supreme Balloon,200,2018-11-11 04:36:46.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3...",75


In [266]:
t = pd.Series(df2['ts'], index=df2.index)
type(t)

pandas.core.series.Series

In [ ]:
column_labels = ["timestamp", "hour", "day", "weekofyear", "month", "year", "weekday"]

In [267]:
# Extract the timestamp, hour, day, week of year, month, year, and weekday from the ts column and set time_data to 
# a list containing these values in order
time_data = []
for data in t:
    time_data.append([data, data.hour, data.day, data.weekofyear, data.month, data.year, data.day_name()])

In [268]:
time_data[0:5]

[[Timestamp('2018-11-11 02:33:56.796000'), 2, 11, 45, 11, 2018, 'Sunday'],
 [Timestamp('2018-11-11 02:36:10.796000'), 2, 11, 45, 11, 2018, 'Sunday'],
 [Timestamp('2018-11-11 02:40:34.796000'), 2, 11, 45, 11, 2018, 'Sunday'],
 [Timestamp('2018-11-11 04:36:13.796000'), 4, 11, 45, 11, 2018, 'Sunday'],
 [Timestamp('2018-11-11 04:36:46.796000'), 4, 11, 45, 11, 2018, 'Sunday']]

In [269]:
type(time_data)

list

In [273]:
time_df = pd.DataFrame.from_records(data = time_data, columns = column_labels)
time_df.head()

,timestamp,hour,day,weekofyear,month,year,weekday
0,2018-11-11 02:33:56.796,2,11,45,11,2018,Sunday
1,2018-11-11 02:36:10.796,2,11,45,11,2018,Sunday
2,2018-11-11 02:40:34.796,2,11,45,11,2018,Sunday
3,2018-11-11 04:36:13.796,4,11,45,11,2018,Sunday
4,2018-11-11 04:36:46.796,4,11,45,11,2018,Sunday


#### Insert Records into Time Table
Implement the time_table_insert query in sql_queries.py and run the cell below to insert records for the timestamps in this log file into the time table. Remember to run create_tables.py before running the cell below to ensure you've created/resetted the time table in the sparkify database.

In [294]:
for index, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

### 4: users Table
Extract Data for Users Table
Select columns for user ID, first name, last name, gender and level and set to user_df

In [304]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73 entries, 0 to 94
Data columns (total 18 columns):
artist           73 non-null object
auth             73 non-null object
firstName        73 non-null object
gender           73 non-null object
itemInSession    73 non-null int64
lastName         73 non-null object
length           73 non-null float64
level            73 non-null object
location         73 non-null object
method           73 non-null object
page             73 non-null object
registration     73 non-null float64
sessionId        73 non-null int64
song             73 non-null object
status           73 non-null int64
ts               73 non-null datetime64[ns]
userAgent        73 non-null object
userId           73 non-null object
dtypes: datetime64[ns](1), float64(2), int64(3), object(12)
memory usage: 13.3+ KB


In [366]:
user_df = df2[['userId', 'firstName', 'lastName', 'gender', 'level']]

In [380]:
# total rows in dataframe
len(user_df)

73

In [382]:
# only 15 user_id uniques will be inserted in table
len(user_df.userId.unique())

15

#### Insert Records into Users Table
Implement the user_table_insert query in sql_queries.py and run the cell below to insert records for the users in this log file into the users table. Remember to run create_tables.py before running the cell below to ensure you've created/resetted the users table in the sparkify database.

In [368]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

In [397]:
cur.execute("SELECT COUNT(*) FROM public.users")
count = cur.fetchone()[0]
print(count, "records succefully inserted")

15 records succefully inserted


## 5: songplays Table
Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the songplays table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.

* Implement the song_select query in sql_queries.py to find the song ID and artist ID based on the title, artist name, and duration of a song.
* Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to songplay_data

In [398]:
df2

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,2018-11-11 02:33:56.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,2018-11-11 02:36:10.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,2018-11-11 02:40:34.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,2018-11-11 04:36:13.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
5,Matmos,Logged In,Joseph,M,0,Gutierrez,1449.11628,free,"Columbia, SC",PUT,NextSong,1.540809e+12,284,Supreme Balloon,200,2018-11-11 04:36:46.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3...",75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,Sidewalk Prophets,Logged In,Molly,F,0,Taylor,260.62322,free,"St. Louis, MO-IL",PUT,NextSong,1.540993e+12,464,You Love Me Anyway (Album),200,2018-11-11 21:09:28.796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",35
91,Rise Against,Logged In,Molly,F,1,Taylor,221.17832,free,"St. Louis, MO-IL",PUT,NextSong,1.540993e+12,464,Torches,200,2018-11-11 21:13:48.796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",35
92,K'Naan,Logged In,Molly,F,2,Taylor,220.49914,free,"St. Louis, MO-IL",PUT,NextSong,1.540993e+12,464,Wavin' Flag,200,2018-11-11 21:17:29.796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",35
93,Patrick Jumpen,Logged In,Ryan,M,0,Smith,208.87465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,480,Holiday,200,2018-11-11 23:39:00.796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26


In [402]:
for index, row in df2.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

In [403]:
conn.close()